# Appendix: Testing VI Implementation 

This notebook walks through how to sanity check our VI implementation through statistical simulations and problem simplifications. 

Future work should focus on translating these into automated unit tests.

In [2]:
# Imports 
%load_ext autoreload
%autoreload 2

import torch

import sys 
sys.path.append('..')
from src.statistical_model import SingleFactorCFA
from src.variational_family import SingleCFAVariationalFamily
from src.variational_sem import VIOptimisationParameters, SingleCFAVIModel
from src.analytical_variational_inference.analytical_variational_infernce import single_factor_cfa_mfvb
from src.analysis.sampling import create_sample_from_qvar, SingleCFAVariationalParameters
from src.mcmc.mcmc import single_factor_cfa_mcmc
from torch.distributions import MultivariateNormal as mvn


/Users/vivbear/Documents/data_science_and_ai/variational-inference-single-factor-cfa-thesis/single-cfa-venv-3.10/lib/python3.10/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/", R: "/Users/vivbear/Documents/data_science_and_ai/variational-inference-single-factor-cfa-thesis/notebooks"
  warnings.warn(
/Users/vivbear/Documents/data_science_and_ai/variational-inference-single-factor-cfa-thesis/single-cfa-venv-3.10/lib/python3.10/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "R_SESSION_TMPDIR" redefined by R and overriding existing variable. Current: "/var/folders/rw/17130_512vz5y5_7d1l60_dh0000gn/T//Rtmpxfehae", R: "/var/folders/rw/17130_512vz5y5_7d1l60_dh0000gn/T//RtmpbVuGb2"
  warnings.warn(
/Users/vivbear/Documents/data_science_and_ai/variational-inference-single-factor-cfa-thesis/single-cfa-venv-3.10/lib/python3.10/site-packages/pystan/lookup.py:10: 

In [3]:
# Set seed for reproducibility 
torch.manual_seed(42)

In [4]:
# Simulated Data

N = 300 
M = 3 
nu = torch.tensor([5.0, 6.0, 2.25])
sig2 = torch.tensor([0.565])
lam = torch.tensor([0.75, 1.05])
psi = torch.tensor([0.8, 1.10, 0.66])

#Generate Latent Parameter Values 
eta = torch.randn(N)*torch.sqrt(sig2)

#Concatenate lambda values 
lam1_fixed = torch.tensor([1.0])
lam_full = torch.cat((lam1_fixed, lam))

#Generate y values based on User Input
# yi ~ id Normal(nu + eta_i * lam, diag(psi)), yi /in R^m
#cov:
like_dist_cov = torch.diag(psi) #m*m tensor 
#means: want a n*m vector of means
like_dist_means = torch.matmul(eta.unsqueeze(1), lam_full.unsqueeze(0)) + nu

y_data_sim = mvn(like_dist_means, covariance_matrix= like_dist_cov).rsample() 

In [5]:
#Hyper-parameter values

#sig_2 ~ InvGamma
sig2_shape = torch.tensor([0.5])  
sig2_rate = torch.tensor([0.5])  

#psi ~ iid Inv Gamma for j = 1..m 
psi_shape = torch.tensor([0.5])  
psi_rate = torch.tensor([0.005])  

#nu ~ iid Normal for j = 1...m
nu_sig2 = torch.tensor([100.0])  
nu_mean = torch.tensor([0.0])

#lam_j | psi_j ~ id Normal(mu, sig2*psi_j)
lam_mean = torch.tensor([0.0])
lam_sig2 = torch.tensor([1.0])

hyper_params = {"sig2_shape": sig2_shape, "sig2_rate": sig2_rate, "psi_shape": psi_shape, "psi_rate": psi_rate, "nu_sig2": nu_sig2, "nu_mean": nu_mean, "lam_mean": lam_mean, "lam_sig2": lam_sig2}

# Part 1 - Single Component Tests

In [6]:
VI_OPTIMISATION_PARAMETERS = VIOptimisationParameters(num_iterations = 20_000, relative_error_threshold= 10-4, patience = 100)

## Estimate all variables

In [7]:
degenerates = {}
model = SingleCFAVIModel(y_data = y_data_sim, hyper_params= hyper_params, degenerates = degenerates)

In [ ]:
model.optimize(optimisation_parameters= VI_OPTIMISATION_PARAMETERS, filename = 'tensorboard_runs/simulated_VIModel_test', K = 10, alpha =1)

  1%|          | 140/20000 [00:05<12:40, 26.11it/s]

VI converged at step t =  140


In [10]:
%reload_ext tensorboard
%tensorboard --logdir tensorboard_runs/simulated_VIModel_test_K_10_alpha1/

Reusing TensorBoard on port 6006 (pid 63399), started 0:00:06 ago. (Use '!kill 63399' to kill it.)

## Estimate all parameters 